In [8]:
from tool import Tool

import ollama
import openai
import json

In [9]:
openai.base_url = "http://localhost:11434/v1"
openai.api_key = 'ollama'

client = ollama.Client()

In [10]:
### FUNCTION DEFINITION ###

import ast

def add_numbers(numbers: list[float]) -> float:
    if isinstance(numbers, str):
        numbers = ast.literal_eval(numbers)
    return sum(map(float, numbers))

In [11]:
tools = {
    'add_numbers': Tool(
        definition={
            'type': 'function',
            'function': {
                'name': 'add_numbers',
                'description': 'Adds the values of all numbers in the list `numbers`',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'numbers': {
                            'type': 'list[float]',
                            'description': 'A list of real numbers',
                        },
                    },
                    'required': ['numbers'],
                },
            }},
        function=add_numbers
    ),
}


def tool_defs(): return [x.definition for x in tools.values()]

In [12]:
def generate_tool_outputs(tool_calls):
    tool_outputs = []

    for tool_call in tool_calls:
        function_name = tool_call['function']['name']
        arguments = tool_call['function']['arguments']

        if function_name in tools:
            output = tools[function_name].call(**arguments)
            print(f'Tool output ({function_name}{arguments}): {output}')
            tool_outputs.append({
                'role': 'tool',
                'name': function_name,
                # 'content': str(output),
                'content': json.dumps(f"{function_name}({arguments}): {str(output)}")
            })

    return tool_outputs


In [13]:
def run_assistant(model, message):
    chat_messages = [
        {
            "role": "user",
            "content": message,
        }
    ]

    res = client.chat(
        model=model,
        messages=chat_messages,
        tools=tool_defs(),
    )

    print(json.dumps(res.model_dump_json(), indent=2))

    msg = res['message']

    # Check if the model called tools
    if msg.get('tool_calls'):
        print(msg['tool_calls'])
        tool_outputs = generate_tool_outputs(msg['tool_calls'])
        # chat_messages.append(msg)
        chat_messages.extend(tool_outputs)

        for chunk in client.chat(
            model=model,
            messages=chat_messages,
            stream=True
        ):
            print(chunk['message']['content'], end='', flush=True)

    else:
        print(msg['content'])

In [14]:
run_assistant("qwen2.5:3b", "What's 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 999 + 19241284176?")

"{\"model\":\"qwen2.5:3b\",\"created_at\":\"2025-05-28T22:08:13.084423003Z\",\"done\":true,\"done_reason\":\"stop\",\"total_duration\":1872848667,\"load_duration\":26762672,\"prompt_eval_count\":217,\"prompt_eval_duration\":19328304,\"eval_count\":91,\"eval_duration\":1801095579,\"message\":{\"role\":\"assistant\",\"content\":\"\",\"images\":null,\"tool_calls\":[{\"function\":{\"name\":\"add_numbers\",\"arguments\":{\"numbers\":[1,2,3,4,5,6,7,8,9,10,999,19241284176]}}}]}}"
[ToolCall(function=Function(name='add_numbers', arguments={'numbers': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 999, 19241284176]}))]
Tool output (add_numbers{'numbers': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 999, 19241284176]}): 19241285230.0
The sum of the numbers 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 + 999 + 19241284176 is 19241285230.0.